In [1]:
from keras.layers import Input, Dense, Embedding, merge, Convolution2D, MaxPooling2D, Dropout
from sklearn.cross_validation import train_test_split
from keras.layers.core import Reshape, Flatten
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
import numpy as np

Using TensorFlow backend.
/Users/Pikachu/.pyenv/versions/2.7.9/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from gensim.models.word2vec import Word2Vec

model_path="../input/word2vec.gensim.model"
model = Word2Vec.load(model_path)

In [3]:
train_tokenized = np.array([[u"大変",u"申し訳",u"ござる",u"ます",u"ん"],
                           [u"誠に",u"ありがとう",u"ござる",u"ます"]])

print type(train_tokenized[0])
print type(train_tokenized)
print train_tokenized.shape
train_distributed = np.array(map(lambda x: list(model.wv[x]), train_tokenized))
train_y = [[1,0],[0,1]]

<type 'list'>
<type 'numpy.ndarray'>
(2,)


In [4]:
test_tokenized = np.array([[u"誠に",u"申し訳",u"ござる",u"ます",u"ん"],
                           [u"本当に",u"ありがとう",u"ござる",u"ます"]])
test_distributed = np.array(map(lambda x: list(model.wv[x]), test_tokenized))
test_y = [[1,0],[0,1]]

In [5]:
print type(test_distributed[0])
print type(test_distributed)
print test_distributed.shape
print test_distributed[0][0].shape[0]
print len(test_distributed[0])

<type 'list'>
<type 'numpy.ndarray'>
(2,)
50
5


In [6]:
from keras.preprocessing import sequence
sequence_length = 8
train_distributed = sequence.pad_sequences(train_distributed, maxlen=sequence_length, padding="post", truncating="post")
test_distributed = sequence.pad_sequences(test_distributed, maxlen=sequence_length, padding="post", truncating="post")

In [7]:
train_distributed = train_distributed[:,:,:,np.newaxis]
test_distributed = test_distributed[:,:,:,np.newaxis]
print train_distributed.shape
print test_distributed.shape

(2, 8, 50, 1)
(2, 8, 50, 1)


In [8]:
filter_sizes = [2,3,4]
#分散表現の次元
vector_length = test_distributed[0][0].shape[0]
#sequence_length = len(test_distributed[0])
num_filters = 512
nb_epoch = 100
batch_size = 30

In [9]:
inputs = Input(shape=(sequence_length,vector_length,1), dtype='float32')
#embedding = Embedding(output_dim=embedding_dim, input_dim=vocabulary_size, input_length=sequence_length)(inputs)
#reshape = Reshape((sequence_length,embedding_dim,1))(embedding)

conv_0 = Convolution2D(num_filters, filter_sizes[0], vector_length, border_mode='valid', init='normal', activation='relu', dim_ordering='tf')(inputs)
conv_1 = Convolution2D(num_filters, filter_sizes[1], vector_length, border_mode='valid', init='normal', activation='relu', dim_ordering='tf')(inputs)
conv_2 = Convolution2D(num_filters, filter_sizes[2], vector_length, border_mode='valid', init='normal', activation='relu', dim_ordering='tf')(inputs)

maxpool_0 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), border_mode='valid', dim_ordering='tf')(conv_0)
maxpool_1 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), border_mode='valid', dim_ordering='tf')(conv_1)
maxpool_2 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), border_mode='valid', dim_ordering='tf')(conv_2)

merged_tensor = merge([maxpool_0, maxpool_1, maxpool_2], mode='concat', concat_axis=1)
flatten = Flatten()(merged_tensor)

In [10]:
output = Dense(output_dim=2, activation='softmax')(flatten)
model = Model(input=inputs, output=output)

checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_distributed, train_y, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1, callbacks=[checkpoint], validation_data=(test_distributed, test_y))

Train on 2 samples, validate on 2 samples
Epoch 1/100
Epoch 00000: val_acc improved from -inf to 0.50000, saving model to weights.000-0.5000.hdf5
2/2 [==============================] - 0s - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/100
Epoch 00001: val_acc did not improve
2/2 [==============================] - 0s - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 3/100
Epoch 00002: val_acc did not improve
2/2 [==============================] - 0s - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 4/100
Epoch 00003: val_acc did not improve
2/2 [==============================] - 0s - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 5/100
Epoch 00004: val_acc did not improve
2/2 [==============================] - 0s - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 6/100
Epoch 00005: val_acc did not improve
2/2 [==============================] - 0s - loss: 0.6931 - acc: 0.500

In [11]:
model.predict(test_distributed)

array([[ 0.5,  0.5],
       [ 0.5,  0.5]], dtype=float32)